In [255]:
import pandas as pd
import numpy as np
import time 
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select

In [249]:
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 10)

In [250]:
driver.get('https://convention2.allacademic.com/one/ica/ica04/')

In [251]:
def click_on_search_papers():
    search_papers = wait.until(EC.element_to_be_clickable((
        By.CSS_SELECTOR, 
        "div.menu_item__icon_text_window__text > a.mainmenu_text"
    )))
    search_papers.click()

In [252]:
click_on_search_papers()

### Iterator

In [253]:
# nth-child(1) means there are two div.iterators, I only need one of the two
iterators = wait.until(EC.presence_of_all_elements_located((
    By.CSS_SELECTOR, "div.iterator:nth-child(1) > form a.fieldtext"
)))

In [254]:
current_page = driver.find_element(
    By.CSS_SELECTOR, 'div.iterator:nth-child(1) > form font.warningtext'
).text

In [255]:
current_page

'1'

In [256]:
for i in iterators:
    print(i.text)

2
3
4
5
6
7
8
9
10
11
12
13
22
Next


In [257]:
print(f'Current page: {current_page}')

Current page: 1


In [258]:
# in the last page, the last element of iterator won't be 'Next'
while iterators[-1].get_attribute('innerHTML') == 'Next':
    current_page = driver.find_element(
        By.CSS_SELECTOR, 'div.iterator:nth-child(1) > form font.warningtext'
    ).text
    print(f'Current page: {current_page}')
    # click on next!
    iterators[-1].click()
    # nth-child(1) means there are two div.iterators, I only need one of the two
    iterators = wait.until(EC.presence_of_all_elements_located((
        By.CSS_SELECTOR, "div.iterator:nth-child(1) > form a.fieldtext"
    )))

Current page: 1
Current page: 2
Current page: 3
Current page: 4
Current page: 5
Current page: 6
Current page: 7
Current page: 8
Current page: 9
Current page: 10
Current page: 11
Current page: 12
Current page: 13
Current page: 14
Current page: 15
Current page: 16
Current page: 17
Current page: 18
Current page: 19
Current page: 20
Current page: 21


## Papers 

In [42]:
papers = driver.find_elements(
    By.CSS_SELECTOR, 'tr.worksheet_window__row__light, tr.worksheet_window__row__dark'
)

In [43]:
first = papers[7]

In [44]:
action = first.find_element(
    By.CSS_SELECTOR, 'td[title="Action"]'
)
paper_link_e = action.find_element(
            By.CSS_SELECTOR, "li.action_list > a.fieldtext"
        )
paper_link = paper_link_e.get_attribute('href')

In [45]:
paper_link

'https://convention2.allacademic.com/one/ica/ica04/index.php?click_key=2&cmd=Multi+Search+Search+Load+Publication&publication_id=112936&PHPSESSID=j704r89v3p8puuqs2lqa7jtiv3'

In [46]:
driver.execute_script("window.open('');")
driver.switch_to.window(driver.window_handles[1])
driver.get(paper_link)

In [47]:
# note that authors_e will return a list since there might be multiple authors
authors_e = driver.find_elements(
    By.CSS_SELECTOR, 'a.search_fieldtext_name'
)

In [48]:
len(authors_e)

5

In [49]:
# authors[0].get_attribute('innerHTML')
for author in authors_e:
    print(author.text)

Robin Nabi (U of Arizona)
Keli L. Finnerty (University of Arizona)
Carmen R. Stitt (University of Arizona)
Jeff Halford (University of Arizona)
Alyssa Quintero (University of Arizona)


In [50]:
# abstract
abstract = driver.find_element(
    By.CSS_SELECTOR, 'blockquote.tight > font.fieldtext'
).get_attribute('innerHTML')

In [51]:
abstract

'This paper reports the results of two studies designed to compare predictors of enjoyment of reality-based and fictional TV programming. In Study 1, 260 adults completed a survey of their cognitive and emotional reactions to either the fictional or reality-based programming that they generally watch. In Study 2, 502 adults completed a similar survey but with a particular reality-based or fictional program in mind. Results of both studies suggest voyeurism (i.e., curiosity about others) is likely a distinguishing marker of reality-based and fictional programming. However, different predictors of enjoyment for reality-based and fictional programming suggests there are as many differences among types of reality-based programming as between reality-based versus fictional programming. Both studies also found more variance explained for the enjoyment of reality-based versus fictional programming. Further, certain gratifications (learning, surprise, suspense, and self-awareness) had quite di

## Integration

In [183]:
def click_on_search_papers():
    """
    Click on 'Search Papers' in the beginning
    """
    search_papers = wait.until(EC.element_to_be_clickable((
        By.CSS_SELECTOR, 
        "div.menu_item__icon_text_window__text > a.mainmenu_text"
    )))
    search_papers.click()

In [135]:
click_on_search_papers()

In [184]:
def get_papers():
    """
    get all paper elements in the current page
    """
    papers = driver.find_elements(
        By.CSS_SELECTOR, 'tr.worksheet_window__row__light, tr.worksheet_window__row__dark'
    )
    return papers

In [137]:
papers = get_papers()

In [138]:
paper = papers[7]

In [139]:
paper_meta_dict_list = []

In [140]:
dic1 = {
    'Index': 1
}
paper_meta_dict_list.append(dic1)
dic2 = {
    'Index': 2
}
paper_meta_dict_list.append(dic2)

In [141]:
paper_meta_dict_list[-1]

{'Index': 2}

In [142]:
paper_meta_dict_list = []

In [185]:
def get_paper_meta(paper, paper_meta_dict_list):
    """
    get paper index, paper title, and paper_type
        the author names can be found here but I'll collect later in the view page
    """
    idx = paper.find_element(
        By.CSS_SELECTOR, 'td[title="##"]').text
    # summary elements:
    summary = paper.find_element(
        By.CSS_SELECTOR, 'td[title="Summary"]'
    )
    title = summary.find_element(
        By.CSS_SELECTOR, 'a.search_headingtext'
    ).text
#     author_names = summary.find_elements(
#         By.CSS_SELECTOR, 'a.search_fieldtext_name'
#     )
    submission_type = summary.find_element(
        By.CSS_SELECTOR, 'td[style="padding: 5px;"]'
    ).text
    paper_meta_dict = {
        'Index': idx,
        'Title': title,
        'Type': submission_type
    }
    # update the dict list
    paper_meta_dict_list.append(paper_meta_dict)
    return paper_meta_dict

In [144]:
paper_meta_dict = get_paper_meta(papers[7], paper_meta_dict_list)

In [145]:
paper_meta_dict_list

[{'Index': '8',
  'Title': 'A comparison of cognitive and emotional predictors of the enjoyment of reality-based and fictional television programming',
  'Type': '  Individual Submission type: Paper'}]

In [186]:
def open_view(paper):
    """
    Input:
        paper element
    Aim:
        open a new window and click 'view'
    """
    action = paper.find_element(
        By.CSS_SELECTOR, 'td[title="Action"]'
    )
    view_link_e = action.find_element(
                By.CSS_SELECTOR, "li.action_list > a.fieldtext"
            )
    view_link = view_link_e.get_attribute('href')
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(view_link)

In [147]:
open_view(paper)

In [187]:
def get_title_to_check(paper_meta_dict_list):
    # there are two 'tr.header font.headingtext'
    # title is the second one
    headingtexts = driver.find_elements(
        By.CSS_SELECTOR, 'tr.header font.headingtext'
    )
    title_to_check = headingtexts[1].text
    # update the most recent paper_meta_dict_list
    paper_meta_dict_list[-1]['Title to Check'] = title_to_check
    return title_to_check

In [149]:
get_title_to_check(paper_meta_dict_list)

'A comparison of cognitive and emotional predictors of the enjoyment of reality-based and fictional television programming'

In [150]:
author_dict_list = []

In [193]:
def get_authors(paper_meta_dict, author_dict_list):
    idx, title = paper_meta_dict['Index'], paper_meta_dict['Title']
    # note that authors_e will return a list since there might be multiple authors
    authors = driver.find_elements(
        By.CSS_SELECTOR, 'a.search_fieldtext_name'
    )
    for author in authors:
        author_idx = authors.index(author) + 1
        authorNum = len(authors)
        author_elements = author.text.split(' (')
        author_name = author_elements[0]
        # doc: https://docs.python.org/3.4/library/stdtypes.html?highlight=strip#str.rstrip
        # some don't contain '()', i.e., affiliation info
        try:
            author_aff = author_elements[1].rstrip(')')
        except:
            author_aff = np.nan
        author_dict = {
            'Paper Index': idx,
            'Paper Title': title,
            'Number of Authors': authorNum,
            'Author Position': author_idx,
            'Author Name': author_name,
            'Author Affiliation': author_aff,
        }
        author_dict_list.append(author_dict)

In [154]:
get_authors(paper_meta_dict, author_dict_list)

In [189]:
def get_abstract(paper_meta_dict_list):
    # abstract
    abstract = driver.find_element(
        By.CSS_SELECTOR, 'blockquote.tight > font.fieldtext'
    ).text
    paper_meta_dict_list[-1]['Abstract'] = abstract
    return abstract

In [156]:
get_abstract(paper_meta_dict_list)

'This paper reports the results of two studies designed to compare predictors of enjoyment of reality-based and fictional TV programming. In Study 1, 260 adults completed a survey of their cognitive and emotional reactions to either the fictional or reality-based programming that they generally watch. In Study 2, 502 adults completed a similar survey but with a particular reality-based or fictional program in mind. Results of both studies suggest voyeurism (i.e., curiosity about others) is likely a distinguishing marker of reality-based and fictional programming. However, different predictors of enjoyment for reality-based and fictional programming suggests there are as many differences among types of reality-based programming as between reality-based versus fictional programming. Both studies also found more variance explained for the enjoyment of reality-based versus fictional programming. Further, certain gratifications (learning, surprise, suspense, and self-awareness) had quite di

In [160]:
pd.DataFrame(paper_meta_dict_list)

,Index,Title,Type,Title to Check,Abstract
0,8,A comparison of cognitive and emotional predic...,Individual Submission type: Paper,A comparison of cognitive and emotional predic...,This paper reports the results of two studies ...


In [161]:
pd.DataFrame(author_dict_list)

,Paper Index,Paper Title,Number of Authors,Author Position,Author Name
0,8,A comparison of cognitive and emotional predic...,5,1,Robin Nabi (U of Arizona)
1,8,A comparison of cognitive and emotional predic...,5,2,Keli L. Finnerty (University of Arizona)
2,8,A comparison of cognitive and emotional predic...,5,3,Carmen R. Stitt (University of Arizona)
3,8,A comparison of cognitive and emotional predic...,5,4,Jeff Halford (University of Arizona)
4,8,A comparison of cognitive and emotional predic...,5,5,Alyssa Quintero (University of Arizona)


## Tring really

In [331]:
def click_on_search_papers():
    search_papers = wait.until(EC.element_to_be_clickable((
        By.CSS_SELECTOR, 
        "div.menu_item__icon_text_window__text > a.mainmenu_text"
    )))
    search_papers.click()

def get_papers():
    """
    get all paper elements in the current page
    """
    papers = driver.find_elements(
        By.CSS_SELECTOR, 'tr.worksheet_window__row__light, tr.worksheet_window__row__dark'
    )
    return papers

def get_paper_meta(paper, year, paper_meta_dict_list):
    """
    get paper index, paper title, and paper_type
        the author names can be found here but I'll collect later in the view page
    """
    idx = paper.find_element(
        By.CSS_SELECTOR, 'td[title="##"]').text
    paper_id = year + '-' + idx
    # summary elements:
    summary = paper.find_element(
        By.CSS_SELECTOR, 'td[title="Summary"]'
    )
    title = summary.find_element(
        By.CSS_SELECTOR, 'a.search_headingtext'
    ).text
    submission_type = summary.find_element(
        By.CSS_SELECTOR, 'td[style="padding: 5px;"]'
    ).text
    paper_meta_dict = {
        'Paper ID': paper_id,
        'Title': title,
        'Type': submission_type
    }
    # update the dict list
    paper_meta_dict_list.append(paper_meta_dict)
    return paper_meta_dict

def open_view(paper):
    """
    Input:
        paper element
    Aim:
        open a new window and click 'view'
    """
    action = paper.find_element(
        By.CSS_SELECTOR, 'td[title="Action"]'
    )
    view_link_e = action.find_element(
                By.CSS_SELECTOR, "li.action_list > a.fieldtext"
            )
    view_link = view_link_e.get_attribute('href')
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(view_link)

def get_title_to_check(paper_meta_dict_list):
    # there are two 'tr.header font.headingtext'
    # title is the second one
    headingtexts = driver.find_elements(
        By.CSS_SELECTOR, 'tr.header font.headingtext'
    )
    title_to_check = headingtexts[1].text
    # update the most recent paper_meta_dict_list
    paper_meta_dict_list[-1]['Title to Check'] = title_to_check
    return title_to_check


def get_authors(paper_meta_dict, author_dict_list):
    paper_id, title = paper_meta_dict['Paper ID'], paper_meta_dict['Title']
    # note that authors_e will return a list since there might be multiple authors
    authors = driver.find_elements(
        By.CSS_SELECTOR, 'a.search_fieldtext_name'
    )
    for author in authors:
        author_idx = authors.index(author) + 1
        authorNum = len(authors)
        author_elements = author.text.split(' (')
        author_name = author_elements[0]
        # doc: https://docs.python.org/3.4/library/stdtypes.html?highlight=strip#str.rstrip
        # some don't contain '()', i.e., affiliation info
        try:
            author_aff = author_elements[1].rstrip(')')
        except:
            author_aff = np.nan
        author_dict = {
            'Paper ID': paper_id,
            'Paper Title': title,
            'Number of Authors': authorNum,
            'Author Position': author_idx,
            'Author Name': author_name,
            'Author Affiliation': author_aff,
        }
        author_dict_list.append(author_dict)

def get_abstract(paper_meta_dict_list):
    # abstract
    abstract = driver.find_element(
        By.CSS_SELECTOR, 'blockquote.tight > font.fieldtext'
    ).text
    paper_meta_dict_list[-1]['Abstract'] = abstract
    return abstract

def scrape_one_page(year, page_num):
    papers = get_papers()
    for paper in papers[0:1]:
        paper_idx = papers.index(paper) + 1
        paper_meta_dict = get_paper_meta(paper, year, paper_meta_dict_list)
        open_view(paper)
        get_title_to_check(paper_meta_dict_list)
        get_authors(paper_meta_dict, author_dict_list)
        get_abstract(paper_meta_dict_list)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        print(f'Page {page_num} Paper {paper_idx} is done')
        time.sleep(0.5)

In [332]:
# # nth-child(1) means there are two div.iterators, I only need one of the two
def get_iterators():
    iterators = driver.find_elements(
        By.CSS_SELECTOR, "div.iterator:nth-child(1) > form a.fieldtext"
    )
    return iterators

In [333]:
first_two_years = [
    'https://convention2.allacademic.com/one/ica/ica03/',
    'https://convention2.allacademic.com/one/ica/ica04/'
]

In [334]:
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 10)

In [335]:
for idx,url in enumerate(first_two_years):
    year = '2003' if idx == 0 else '2004'
    driver.get(url)
    click_on_search_papers()
    paper_meta_dict_list = []
    author_dict_list = []
    iterators = get_iterators()
    total_pages = int(iterators[-2].text)
    for i in range(1,total_pages+1):
        page_num = i
        if i >= 10:
            if year == '2004':
                select = Select(driver.find_element(
                    By.XPATH, '//child::div[@class="iterator"][1] // select'
                ))
                select.select_by_visible_text('+ 20')
#                 iterators = get_iterators()
            else:
                iterators = get_iterators()
                # if '2003', click on '20'
                iterators[-2].click()
        iterators = get_iterators()
        for j in iterators:
            if (j.text == str(i)):
                j.click()
                break 
        scrape_one_page(year,page_num)
        print(f'page {i} is done')
        if i < total_pages - 1:
            driver.refresh()
        if i == total_pages - 1:
            iterators = get_iterators()
            iterators[1].click()
    print(f'{year} is done!')
    paper_df = pd.DataFrame(paper_meta_dict_list)
    author_df = pd.DataFrame(author_dict_list)
    paper_df.to_csv(f'../data/interim/paper_df_{year}.csv', index = False)
    author_df.to_csv(f'../data/interim/author_df_{year}.csv', index = False)
driver.close()
driver.quit()

Page 1 Paper 1 is done
Page 1 Paper 2 is done
Page 1 Paper 3 is done
Page 1 Paper 4 is done
Page 1 Paper 5 is done
Page 1 Paper 6 is done
Page 1 Paper 7 is done
Page 1 Paper 8 is done
Page 1 Paper 9 is done
Page 1 Paper 10 is done
Page 1 Paper 11 is done
Page 1 Paper 12 is done
Page 1 Paper 13 is done
Page 1 Paper 14 is done
Page 1 Paper 15 is done
Page 1 Paper 16 is done
Page 1 Paper 17 is done
Page 1 Paper 18 is done
Page 1 Paper 19 is done
Page 1 Paper 20 is done
Page 1 Paper 21 is done
Page 1 Paper 22 is done
Page 1 Paper 23 is done
Page 1 Paper 24 is done
Page 1 Paper 25 is done
Page 1 Paper 26 is done
Page 1 Paper 27 is done
Page 1 Paper 28 is done
Page 1 Paper 29 is done
Page 1 Paper 30 is done
Page 1 Paper 31 is done
Page 1 Paper 32 is done
Page 1 Paper 33 is done
Page 1 Paper 34 is done
Page 1 Paper 35 is done
Page 1 Paper 36 is done
Page 1 Paper 37 is done
Page 1 Paper 38 is done
Page 1 Paper 39 is done
Page 1 Paper 40 is done
Page 1 Paper 41 is done
Page 1 Paper 42 is done
P

Page 7 Paper 42 is done
Page 7 Paper 43 is done
Page 7 Paper 44 is done
Page 7 Paper 45 is done
Page 7 Paper 46 is done
Page 7 Paper 47 is done
Page 7 Paper 48 is done
Page 7 Paper 49 is done
Page 7 Paper 50 is done
page 7 is done
Page 8 Paper 1 is done
Page 8 Paper 2 is done
Page 8 Paper 3 is done
Page 8 Paper 4 is done
Page 8 Paper 5 is done
Page 8 Paper 6 is done
Page 8 Paper 7 is done
Page 8 Paper 8 is done
Page 8 Paper 9 is done
Page 8 Paper 10 is done
Page 8 Paper 11 is done
Page 8 Paper 12 is done
Page 8 Paper 13 is done
Page 8 Paper 14 is done
Page 8 Paper 15 is done
Page 8 Paper 16 is done
Page 8 Paper 17 is done
Page 8 Paper 18 is done
Page 8 Paper 19 is done
Page 8 Paper 20 is done
Page 8 Paper 21 is done
Page 8 Paper 22 is done
Page 8 Paper 23 is done
Page 8 Paper 24 is done
Page 8 Paper 25 is done
Page 8 Paper 26 is done
Page 8 Paper 27 is done
Page 8 Paper 28 is done
Page 8 Paper 29 is done
Page 8 Paper 30 is done
Page 8 Paper 31 is done
Page 8 Paper 32 is done
Page 8 Pap

Page 14 Paper 23 is done
Page 14 Paper 24 is done
Page 14 Paper 25 is done
Page 14 Paper 26 is done
Page 14 Paper 27 is done
Page 14 Paper 28 is done
Page 14 Paper 29 is done
Page 14 Paper 30 is done
Page 14 Paper 31 is done
Page 14 Paper 32 is done
Page 14 Paper 33 is done
Page 14 Paper 34 is done
Page 14 Paper 35 is done
Page 14 Paper 36 is done
Page 14 Paper 37 is done
Page 14 Paper 38 is done
Page 14 Paper 39 is done
Page 14 Paper 40 is done
Page 14 Paper 41 is done
Page 14 Paper 42 is done
Page 14 Paper 43 is done
Page 14 Paper 44 is done
Page 14 Paper 45 is done
Page 14 Paper 46 is done
Page 14 Paper 47 is done
Page 14 Paper 48 is done
Page 14 Paper 49 is done
Page 14 Paper 50 is done
page 14 is done
Page 15 Paper 1 is done
Page 15 Paper 2 is done
Page 15 Paper 3 is done
Page 15 Paper 4 is done
Page 15 Paper 5 is done
Page 15 Paper 6 is done
Page 15 Paper 7 is done
Page 15 Paper 8 is done
Page 15 Paper 9 is done
Page 15 Paper 10 is done
Page 15 Paper 11 is done
Page 15 Paper 12 i

Page 1 Paper 5 is done
Page 1 Paper 6 is done
Page 1 Paper 7 is done
Page 1 Paper 8 is done
Page 1 Paper 9 is done
Page 1 Paper 10 is done
Page 1 Paper 11 is done
Page 1 Paper 12 is done
Page 1 Paper 13 is done
Page 1 Paper 14 is done
Page 1 Paper 15 is done
Page 1 Paper 16 is done
Page 1 Paper 17 is done
Page 1 Paper 18 is done
Page 1 Paper 19 is done
Page 1 Paper 20 is done
Page 1 Paper 21 is done
Page 1 Paper 22 is done
Page 1 Paper 23 is done
Page 1 Paper 24 is done
Page 1 Paper 25 is done
Page 1 Paper 26 is done
Page 1 Paper 27 is done
Page 1 Paper 28 is done
Page 1 Paper 29 is done
Page 1 Paper 30 is done
Page 1 Paper 31 is done
Page 1 Paper 32 is done
Page 1 Paper 33 is done
Page 1 Paper 34 is done
Page 1 Paper 35 is done
Page 1 Paper 36 is done
Page 1 Paper 37 is done
Page 1 Paper 38 is done
Page 1 Paper 39 is done
Page 1 Paper 40 is done
Page 1 Paper 41 is done
Page 1 Paper 42 is done
Page 1 Paper 43 is done
Page 1 Paper 44 is done
Page 1 Paper 45 is done
Page 1 Paper 46 is do

Page 7 Paper 46 is done
Page 7 Paper 47 is done
Page 7 Paper 48 is done
Page 7 Paper 49 is done
Page 7 Paper 50 is done
page 7 is done
Page 8 Paper 1 is done
Page 8 Paper 2 is done
Page 8 Paper 3 is done
Page 8 Paper 4 is done
Page 8 Paper 5 is done
Page 8 Paper 6 is done
Page 8 Paper 7 is done
Page 8 Paper 8 is done
Page 8 Paper 9 is done
Page 8 Paper 10 is done
Page 8 Paper 11 is done
Page 8 Paper 12 is done
Page 8 Paper 13 is done
Page 8 Paper 14 is done
Page 8 Paper 15 is done
Page 8 Paper 16 is done
Page 8 Paper 17 is done
Page 8 Paper 18 is done
Page 8 Paper 19 is done
Page 8 Paper 20 is done
Page 8 Paper 21 is done
Page 8 Paper 22 is done
Page 8 Paper 23 is done
Page 8 Paper 24 is done
Page 8 Paper 25 is done
Page 8 Paper 26 is done
Page 8 Paper 27 is done
Page 8 Paper 28 is done
Page 8 Paper 29 is done
Page 8 Paper 30 is done
Page 8 Paper 31 is done
Page 8 Paper 32 is done
Page 8 Paper 33 is done
Page 8 Paper 34 is done
Page 8 Paper 35 is done
Page 8 Paper 36 is done
Page 8 Pap

Page 14 Paper 27 is done
Page 14 Paper 28 is done
Page 14 Paper 29 is done
Page 14 Paper 30 is done
Page 14 Paper 31 is done
Page 14 Paper 32 is done
Page 14 Paper 33 is done
Page 14 Paper 34 is done
Page 14 Paper 35 is done
Page 14 Paper 36 is done
Page 14 Paper 37 is done
Page 14 Paper 38 is done
Page 14 Paper 39 is done
Page 14 Paper 40 is done
Page 14 Paper 41 is done
Page 14 Paper 42 is done
Page 14 Paper 43 is done
Page 14 Paper 44 is done
Page 14 Paper 45 is done
Page 14 Paper 46 is done
Page 14 Paper 47 is done
Page 14 Paper 48 is done
Page 14 Paper 49 is done
Page 14 Paper 50 is done
page 14 is done
Page 15 Paper 1 is done
Page 15 Paper 2 is done
Page 15 Paper 3 is done
Page 15 Paper 4 is done
Page 15 Paper 5 is done
Page 15 Paper 6 is done
Page 15 Paper 7 is done
Page 15 Paper 8 is done
Page 15 Paper 9 is done
Page 15 Paper 10 is done
Page 15 Paper 11 is done
Page 15 Paper 12 is done
Page 15 Paper 13 is done
Page 15 Paper 14 is done
Page 15 Paper 15 is done
Page 15 Paper 16 i

Page 21 Paper 3 is done
Page 21 Paper 4 is done
Page 21 Paper 5 is done
Page 21 Paper 6 is done
Page 21 Paper 7 is done
Page 21 Paper 8 is done
Page 21 Paper 9 is done
Page 21 Paper 10 is done
Page 21 Paper 11 is done
Page 21 Paper 12 is done
Page 21 Paper 13 is done
Page 21 Paper 14 is done
Page 21 Paper 15 is done
Page 21 Paper 16 is done
Page 21 Paper 17 is done
Page 21 Paper 18 is done
Page 21 Paper 19 is done
Page 21 Paper 20 is done
Page 21 Paper 21 is done
Page 21 Paper 22 is done
Page 21 Paper 23 is done
Page 21 Paper 24 is done
Page 21 Paper 25 is done
Page 21 Paper 26 is done
Page 21 Paper 27 is done
Page 21 Paper 28 is done
Page 21 Paper 29 is done
Page 21 Paper 30 is done
Page 21 Paper 31 is done
Page 21 Paper 32 is done
Page 21 Paper 33 is done
Page 21 Paper 34 is done
Page 21 Paper 35 is done
Page 21 Paper 36 is done
Page 21 Paper 37 is done
Page 21 Paper 38 is done
Page 21 Paper 39 is done
Page 21 Paper 40 is done
Page 21 Paper 41 is done
Page 21 Paper 42 is done
Page 21

In [327]:
pd.DataFrame(paper_meta_dict_list)

,Paper ID,Title,Type,Title to Check,Abstract
0,2004-1,A Bibliometric Analysis of Disciplinary Self-R...,Individual Submission type: Paper,A Bibliometric Analysis of Disciplinary Self-R...,This paper reports the results of a bibliometr...
1,2004-2,A Big Fat Indie Story: Discourses Surrounding ...,Individual Submission type: Paper,A Big Fat Indie Story: Discourses Surrounding ...,"In this paper, I argue that the standard media..."
2,2004-51,A Meta-analysis of Journal Articles Intersecti...,Individual Submission type: Paper,A Meta-analysis of Journal Articles Intersecti...,"During the past several years, the propagation..."
3,2004-52,A Methodology for Observing Emotions During In...,Individual Submission type: Paper,A Methodology for Observing Emotions During In...,Although face to face arguing is a kind of com...
4,2004-101,A smoker is never alone: Test of a typology of...,Individual Submission type: Paper,A smoker is never alone: Test of a typology of...,The operationalization of social influence in ...
5,2004-102,A Social-Biological Perspective on the News,Individual Submission type: Paper,A Social-Biological Perspective on the News,This paper elaborates an evolutionary psycholo...
6,2004-151,Can U.S. Entertainment TV Programs InfluenceTh...,Individual Submission type: Paper,Can U.S. Entertainment TV Programs InfluenceTh...,Shortly after the September 11 terrorist attac...
7,2004-152,Cave Art and the Origins of Visual Communication,Individual Submission type: Paper,Cave Art and the Origins of Visual Communication,"Using a constructivist approach, this paper bo..."
8,2004-201,Contesting Narratives of Workplace Maternity,Individual Submission type: Paper,Contesting Narratives of Workplace Maternity,Our paper situates discourses and practices of...
9,2004-202,Contexts of Commemoration: A Comparative Study...,Individual Submission type: Paper,Contexts of Commemoration: A Comparative Study...,This paper contrasts and compares two salient ...


In [328]:
pd.DataFrame(author_dict_list)

,Paper ID,Paper Title,Number of Authors,Author Position,Author Name,Author Affiliation
0,2004-1,A Bibliometric Analysis of Disciplinary Self-R...,1,1,William J. White,Penn State Altoona
1,2004-2,A Big Fat Indie Story: Discourses Surrounding ...,1,1,Alisa Hayley Perren,University of Texas at Austin
2,2004-51,A Meta-analysis of Journal Articles Intersecti...,1,1,Cindy Louise Royal,"U of Texas, Austin, TX, USA"
3,2004-52,A Methodology for Observing Emotions During In...,1,1,Dale Hample,Western Illinois U
4,2004-101,A smoker is never alone: Test of a typology of...,3,1,Bas van den Putte,U of Amsterdam
...,...,...,...,...,...,...
74,2004-1002,Tracking the Global Footprint of Planet Hollyw...,3,3,Arvind Singhal,Ohio U
75,2004-1051,Virtual Democratic Possibilities: Creating and...,3,1,James S Fishkin,Stanford
76,2004-1051,Virtual Democratic Possibilities: Creating and...,3,2,Shanto Iyengar,Stanford U
77,2004-1051,Virtual Democratic Possibilities: Creating and...,3,3,Robert Luskin,U of Texas
